<a href="https://colab.research.google.com/github/Somtech4/machine-learning-threat-detection/blob/main/api.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# CELL 1 — Install + mount
from google.colab import drive
drive.mount('/content/drive')
!pip install fastapi uvicorn pyngrok -q

Mounted at /content/drive


In [3]:
# CELL 2 — NOW RUN YOUR FINAL FLASK (100% working)
import os, json, numpy as np, pandas as pd, requests
from fastapi import FastAPI
from pyngrok import ngrok
import joblib
import threading
import uvicorn

MODEL_DIR = "/content/models"

rf = joblib.load(f"{MODEL_DIR}/main_model.joblib")
scaler = joblib.load(f"{MODEL_DIR}/main_scaler.joblib")
le = joblib.load(f"{MODEL_DIR}/label_encoder.joblib")
features = json.load(open(f"{MODEL_DIR}/features.json"))
threshold = 0.85

print(f"LOADED — Classes: {list(le.classes_)} — Threshold: {threshold}")

app = FastAPI()

@app.post("/score")
async def score(flow: dict):
    try:
        df = pd.DataFrame([flow])
        for c in features:
            if c not in df.columns:
                df[c] = 0
        df = df[features].fillna(0)
        X = scaler.transform(df.values)

        conf = float(rf.predict_proba(X)[0][1:].sum())
        label = le.inverse_transform(rf.predict(X))[0]
        final = label if conf >= threshold else "BENIGN"

        print(f"→ {final} (confidence={conf:.4f})")

        if final != "BENIGN":
            requests.post(
                "https://172.237.110.64:8088/services/collector",
                headers={"Authorization": "Splunk 97c40840-1287-441b-800f-344bba5ac69e"},
                json={"event": {"final_label": final, "confidence": round(conf,4)}, "index": "lab", "sourcetype": "ml_network_events"},
                verify=False
            )

        return {"final_label": final, "confidence": conf}
    except Exception as e:
        return {"error": str(e)}

ngrok.set_auth_token("361JMyfQWwRa738olJ2oG2crPE0_68T2wTBiiXjDccG1Na4CY")
public_url = ngrok.connect(8000)
print("LIVE →", public_url.public_url + "/score")

threading.Thread(target=uvicorn.run, args=(app,), kwargs={"host": "0.0.0.0", "port": 8000}, daemon=True).start()
print("Server running — test now!")

FileNotFoundError: [Errno 2] No such file or directory: '/content/models/main_model.joblib'

LOADED — Classes: ['Benign', 'Brute Force', 'DDoS', 'Malware', 'Web Attack', 'Zero-Day'] — Threshold: 0.85
LIVE → http://unadjacent-infusorial-jeromy.ngrok-free.dev/score
Server running — test now!
 * Serving Flask app '__main__'
 * Debug mode: off
